# AIRO Group Holdings - Quantitative Equity Analysis
## Comparative Valuation: Aerospace, Defense & eVTOL Sectors

**Analysis Date:** January 8, 2026  
**Analyst Methodology:** Goldman Sachs-style equity research with DCF, Comparable Company Analysis, and Sum-of-Parts valuation

### Objective
This notebook provides comprehensive quantitative analysis of AIRO Group Holdings (NASDAQ: AIRO) against its primary competitors across three segments:
- **Drone/Defense:** AeroVironment (AVAV), Kratos Defense (KTOS)
- **eVTOL:** Joby Aviation (JOBY), Archer Aviation (ACHR)
- **Diversified Comparison:** All peers

### Valuation Methodologies
1. **Trading Comparables Analysis** - EV/Sales, P/B, Price/Revenue multiples
2. **Financial Performance Metrics** - Revenue growth, margin trends, cash burn
3. **Sum-of-Parts (SOTP) Valuation** - Segment-level analysis for AIRO's four divisions
4. **Technical Analysis** - Price momentum, relative strength, volatility
5. **Risk-Adjusted Return Framework** - Sharpe ratios, beta analysis, downside protection

In [ ]:
# Import required libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configure visualization aesthetics for professional presentation
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['legend.fontsize'] = 10

print("✓ Libraries imported successfully")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1. Data Collection & Company Universe

We'll analyze AIRO against five key competitors spanning the drone, defense, and eVTOL markets. Each company represents a different strategic positioning within the aerospace ecosystem.

In [ ]:
# Define company universe with strategic categorization
companies = {
    'AIRO': {'name': 'AIRO Group Holdings', 'category': 'Diversified', 'segment': 'Drone+eVTOL+Training'},
    'AVAV': {'name': 'AeroVironment', 'category': 'Drone/Defense', 'segment': 'Military UAS'},
    'KTOS': {'name': 'Kratos Defense', 'category': 'Drone/Defense', 'segment': 'Unmanned Systems'},
    'JOBY': {'name': 'Joby Aviation', 'category': 'eVTOL', 'segment': 'Air Taxi'},
    'ACHR': {'name': 'Archer Aviation', 'category': 'eVTOL', 'segment': 'Air Taxi'},
}

tickers = list(companies.keys())
print("Company Universe:")
print("="*80)
for ticker, info in companies.items():
    print(f"{ticker:6} | {info['name']:25} | {info['category']:15} | {info['segment']}")
print("="*80)

In [ ]:
# Download historical price data (2 years for trend analysis)
# Using 2-year window to capture IPO performance for AIRO (June 2025) and volatility patterns

end_date = datetime.now()
start_date = end_date - timedelta(days=730)  # 2 years

print(f"Downloading market data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}...")
print()

price_data = {}
stock_info = {}

for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        price_data[ticker] = stock.history(start=start_date, end=end_date)
        stock_info[ticker] = stock.info
        print(f"✓ {ticker:6} - {len(price_data[ticker])} trading days retrieved")
    except Exception as e:
        print(f"✗ {ticker:6} - Error: {str(e)}")
        price_data[ticker] = pd.DataFrame()
        stock_info[ticker] = {}

print("\n✓ Data collection complete")

## 2. Key Financial Metrics Extraction

We'll extract critical valuation metrics from each company, focusing on market capitalization, revenue metrics, profitability indicators, and balance sheet strength. For pre-revenue companies like JOBY and ACHR, we'll use forward-looking estimates where available.

In [ ]:
def safe_get(info_dict, key, default=np.nan):
    """Safely extract values from stock info dictionary, handling missing keys gracefully"""
    return info_dict.get(key, default)

def calculate_ev(market_cap, cash, debt):
    """Calculate Enterprise Value: Market Cap + Debt - Cash"""
    try:
        return market_cap + debt - cash
    except:
        return np.nan

# Build comprehensive financial metrics dataframe
metrics_data = []

for ticker in tickers:
    info = stock_info[ticker]
    
    # Current price from latest close
    current_price = price_data[ticker]['Close'].iloc[-1] if len(price_data[ticker]) > 0 else np.nan
    
    # Market capitalization and enterprise value components
    market_cap = safe_get(info, 'marketCap', np.nan)
    cash = safe_get(info, 'totalCash', 0)
    debt = safe_get(info, 'totalDebt', 0)
    enterprise_value = calculate_ev(market_cap, cash, debt)
    
    # Revenue metrics (TTM = Trailing Twelve Months)
    revenue = safe_get(info, 'totalRevenue', np.nan)
    revenue_growth = safe_get(info, 'revenueGrowth', np.nan)
    
    # Profitability metrics
    gross_margin = safe_get(info, 'grossMargins', np.nan)
    operating_margin = safe_get(info, 'operatingMargins', np.nan)
    profit_margin = safe_get(info, 'profitMargins', np.nan)
    
    # Per-share metrics
    eps = safe_get(info, 'trailingEps', np.nan)
    book_value = safe_get(info, 'bookValue', np.nan)
    
    # Valuation multiples
    pe_ratio = safe_get(info, 'trailingPE', np.nan)
    pb_ratio = safe_get(info, 'priceToBook', np.nan)
    ps_ratio = market_cap / revenue if revenue and revenue > 0 else np.nan
    ev_sales = enterprise_value / revenue if revenue and revenue > 0 else np.nan
    
    # Risk metrics
    beta = safe_get(info, 'beta', np.nan)
    
    # Analyst recommendations
    target_price = safe_get(info, 'targetMeanPrice', np.nan)
    num_analysts = safe_get(info, 'numberOfAnalystOpinions', 0)
    
    metrics_data.append({
        'Ticker': ticker,
        'Company': companies[ticker]['name'],
        'Category': companies[ticker]['category'],
        'Current Price': current_price,
        'Market Cap (M)': market_cap / 1e6 if market_cap else np.nan,
        'Enterprise Value (M)': enterprise_value / 1e6 if enterprise_value else np.nan,
        'Revenue TTM (M)': revenue / 1e6 if revenue else np.nan,
        'Revenue Growth %': revenue_growth * 100 if revenue_growth else np.nan,
        'Gross Margin %': gross_margin * 100 if gross_margin else np.nan,
        'Operating Margin %': operating_margin * 100 if operating_margin else np.nan,
        'Profit Margin %': profit_margin * 100 if profit_margin else np.nan,
        'EPS': eps,
        'Book Value': book_value,
        'P/E': pe_ratio,
        'P/B': pb_ratio,
        'P/S': ps_ratio,
        'EV/Sales': ev_sales,
        'Beta': beta,
        'Analyst Target': target_price,
        'Upside to Target %': ((target_price / current_price - 1) * 100) if target_price and current_price else np.nan,
        'Num Analysts': num_analysts
    })

df_metrics = pd.DataFrame(metrics_data)

# Display formatted metrics table
print("\n" + "="*100)
print("FINANCIAL METRICS SUMMARY")
print("="*100)
print(df_metrics.to_string(index=False))
print("="*100)

## 3. Valuation Analysis - Trading Comparables

We'll analyze AIRO's valuation relative to peers using multiple frameworks. This is the cornerstone of relative valuation methodology used by Goldman Sachs and other bulge bracket investment banks.

In [ ]:
# Create valuation comparison focusing on key multiples
valuation_cols = ['Ticker', 'Company', 'Category', 'Market Cap (M)', 'P/S', 'EV/Sales', 'P/B', 'P/E']
df_valuation = df_metrics[valuation_cols].copy()

# Calculate peer group statistics for relative valuation analysis
print("\n" + "="*100)
print("VALUATION MULTIPLES ANALYSIS")
print("="*100)
print(df_valuation.to_string(index=False))

# Peer group averages by category
print("\n" + "-"*100)
print("PEER GROUP AVERAGES BY CATEGORY")
print("-"*100)

peer_averages = df_metrics.groupby('Category')[['P/S', 'EV/Sales', 'P/B', 'Revenue Growth %', 'Gross Margin %']].mean()
print(peer_averages)

# AIRO's position relative to each peer group
airo_metrics = df_metrics[df_metrics['Ticker'] == 'AIRO'].iloc[0]

print("\n" + "-"*100)
print("AIRO RELATIVE VALUATION POSITIONING")
print("-"*100)

for category in peer_averages.index:
    if category != 'Diversified':
        print(f"\nVs. {category} Peers:")
        for metric in ['P/S', 'EV/Sales']:
            airo_val = airo_metrics[metric]
            peer_avg = peer_averages.loc[category, metric]
            if not np.isnan(airo_val) and not np.isnan(peer_avg):
                discount_premium = ((airo_val / peer_avg) - 1) * 100
                status = "PREMIUM" if discount_premium > 0 else "DISCOUNT"
                print(f"  {metric:12} | AIRO: {airo_val:.2f}x | Peer Avg: {peer_avg:.2f}x | {discount_premium:+.1f}% {status}")

In [ ]:
# Visualization 1: Valuation Multiples Comparison
# This chart provides immediate visual insight into relative valuation positioning

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Comparative Valuation Analysis - AIRO vs. Peers', fontsize=16, fontweight='bold', y=0.995)

# Plot 1: EV/Sales Multiple Comparison
ax1 = axes[0, 0]
df_plot = df_metrics.dropna(subset=['EV/Sales']).sort_values('EV/Sales')
colors = ['#FF4444' if x == 'AIRO' else '#4444FF' for x in df_plot['Ticker']]
bars1 = ax1.barh(df_plot['Ticker'], df_plot['EV/Sales'], color=colors, alpha=0.7, edgecolor='black')
ax1.set_xlabel('EV/Sales Multiple', fontweight='bold')
ax1.set_title('Enterprise Value / Sales Ratio', fontweight='bold')
ax1.grid(axis='x', alpha=0.3)
for i, (ticker, val) in enumerate(zip(df_plot['Ticker'], df_plot['EV/Sales'])):
    ax1.text(val + 0.1, i, f'{val:.2f}x', va='center', fontweight='bold')

# Plot 2: Price/Sales Multiple Comparison
ax2 = axes[0, 1]
df_plot = df_metrics.dropna(subset=['P/S']).sort_values('P/S')
colors = ['#FF4444' if x == 'AIRO' else '#44AA44' for x in df_plot['Ticker']]
bars2 = ax2.barh(df_plot['Ticker'], df_plot['P/S'], color=colors, alpha=0.7, edgecolor='black')
ax2.set_xlabel('P/S Multiple', fontweight='bold')
ax2.set_title('Price / Sales Ratio', fontweight='bold')
ax2.grid(axis='x', alpha=0.3)
for i, (ticker, val) in enumerate(zip(df_plot['Ticker'], df_plot['P/S'])):
    ax2.text(val + 0.1, i, f'{val:.2f}x', va='center', fontweight='bold')

# Plot 3: Market Cap Comparison (Log Scale)
ax3 = axes[1, 0]
df_plot = df_metrics.dropna(subset=['Market Cap (M)']).sort_values('Market Cap (M)')
colors = ['#FF4444' if x == 'AIRO' else '#AA44AA' for x in df_plot['Ticker']]
bars3 = ax3.barh(df_plot['Ticker'], df_plot['Market Cap (M)'], color=colors, alpha=0.7, edgecolor='black')
ax3.set_xlabel('Market Capitalization ($M)', fontweight='bold')
ax3.set_title('Market Capitalization Comparison (Log Scale)', fontweight='bold')
ax3.set_xscale('log')
ax3.grid(axis='x', alpha=0.3, which='both')
for i, (ticker, val) in enumerate(zip(df_plot['Ticker'], df_plot['Market Cap (M)'])):
    ax3.text(val * 1.2, i, f'${val:,.0f}M', va='center', fontsize=9, fontweight='bold')

# Plot 4: Gross Margin vs EV/Sales (Efficiency-Valuation Matrix)
ax4 = axes[1, 1]
df_plot = df_metrics.dropna(subset=['Gross Margin %', 'EV/Sales'])
for idx, row in df_plot.iterrows():
    color = '#FF4444' if row['Ticker'] == 'AIRO' else '#4444FF'
    size = 200 if row['Ticker'] == 'AIRO' else 100
    ax4.scatter(row['Gross Margin %'], row['EV/Sales'], s=size, color=color, alpha=0.6, edgecolor='black')
    ax4.annotate(row['Ticker'], (row['Gross Margin %'], row['EV/Sales']), 
                fontweight='bold', fontsize=10, ha='center', va='bottom')
ax4.set_xlabel('Gross Margin (%)', fontweight='bold')
ax4.set_ylabel('EV/Sales Multiple', fontweight='bold')
ax4.set_title('Profitability vs Valuation Matrix', fontweight='bold')
ax4.grid(alpha=0.3)
ax4.axhline(df_metrics['EV/Sales'].median(), color='gray', linestyle='--', alpha=0.5, label='Median EV/Sales')
ax4.axvline(df_metrics['Gross Margin %'].median(), color='gray', linestyle='--', alpha=0.5, label='Median Gross Margin')
ax4.legend(fontsize=8)

plt.tight_layout()
plt.show()

print("\n✓ Valuation comparison charts generated")

## 4. Stock Price Performance Analysis

Technical analysis of price momentum, relative performance, and volatility patterns. This section provides insight into market sentiment and trading dynamics for each security.

In [ ]:
# Calculate performance metrics for different time periods
def calculate_returns(price_series, periods=[5, 20, 60, 120, 252]):
    """
    Calculate returns over multiple periods (trading days):
    5d = 1 week, 20d = 1 month, 60d = 3 months, 120d = 6 months, 252d = 1 year
    """
    returns = {}
    for period in periods:
        if len(price_series) >= period:
            returns[f'{period}d'] = ((price_series.iloc[-1] / price_series.iloc[-period]) - 1) * 100
        else:
            returns[f'{period}d'] = np.nan
    return returns

def calculate_volatility(price_series, window=20):
    """Calculate annualized volatility from daily returns"""
    if len(price_series) >= window:
        returns = price_series.pct_change().dropna()
        return returns.std() * np.sqrt(252) * 100  # Annualized volatility
    return np.nan

# Build performance metrics dataframe
performance_data = []

for ticker in tickers:
    if len(price_data[ticker]) > 0:
        close_prices = price_data[ticker]['Close']
        returns = calculate_returns(close_prices)
        volatility = calculate_volatility(close_prices, window=60)
        
        # Current price levels
        current_price = close_prices.iloc[-1]
        high_52w = close_prices[-252:].max() if len(close_prices) >= 252 else close_prices.max()
        low_52w = close_prices[-252:].min() if len(close_prices) >= 252 else close_prices.min()
        
        performance_data.append({
            'Ticker': ticker,
            'Company': companies[ticker]['name'],
            'Current Price': current_price,
            '52W High': high_52w,
            '52W Low': low_52w,
            '% Off High': ((current_price / high_52w) - 1) * 100,
            '1W Return %': returns.get('5d', np.nan),
            '1M Return %': returns.get('20d', np.nan),
            '3M Return %': returns.get('60d', np.nan),
            '6M Return %': returns.get('120d', np.nan),
            '1Y Return %': returns.get('252d', np.nan),
            'Volatility % (Ann.)': volatility
        })

df_performance = pd.DataFrame(performance_data)

print("\n" + "="*120)
print("PRICE PERFORMANCE & MOMENTUM ANALYSIS")
print("="*120)
print(df_performance.to_string(index=False))
print("="*120)

In [ ]:
# Visualization 2: Stock Price Charts with Technical Indicators

fig, axes = plt.subplots(3, 2, figsize=(18, 14))
fig.suptitle('Historical Price Performance & Technical Analysis', fontsize=16, fontweight='bold', y=0.995)

for idx, ticker in enumerate(tickers):
    row = idx // 2
    col = idx % 2
    ax = axes[row, col]
    
    if len(price_data[ticker]) > 0:
        df = price_data[ticker].copy()
        
        # Calculate moving averages for trend identification
        df['MA20'] = df['Close'].rolling(window=20).mean()  # Short-term trend
        df['MA50'] = df['Close'].rolling(window=50).mean()  # Medium-term trend
        df['MA200'] = df['Close'].rolling(window=200).mean()  # Long-term trend
        
        # Plot price with moving averages
        color = '#FF4444' if ticker == 'AIRO' else '#4444FF'
        ax.plot(df.index, df['Close'], linewidth=2, color=color, label='Close Price', alpha=0.8)
        ax.plot(df.index, df['MA20'], linewidth=1.5, color='orange', label='MA20', alpha=0.7, linestyle='--')
        ax.plot(df.index, df['MA50'], linewidth=1.5, color='green', label='MA50', alpha=0.7, linestyle='--')
        
        # Add volume on secondary axis
        ax2 = ax.twinx()
        ax2.bar(df.index, df['Volume'], alpha=0.2, color='gray', label='Volume')
        ax2.set_ylabel('Volume', fontsize=9)
        ax2.tick_params(labelsize=8)
        
        # Formatting
        ax.set_title(f"{ticker} - {companies[ticker]['name']}", fontweight='bold', fontsize=12)
        ax.set_ylabel('Price ($)', fontweight='bold')
        ax.legend(loc='upper left', fontsize=8)
        ax.grid(alpha=0.3)
        ax.tick_params(labelsize=8)
        
        # Add performance annotation
        perf_1y = df_performance[df_performance['Ticker'] == ticker]['1Y Return %'].values[0]
        if not np.isnan(perf_1y):
            color_text = 'green' if perf_1y > 0 else 'red'
            ax.text(0.02, 0.98, f'1Y: {perf_1y:+.1f}%', transform=ax.transAxes,
                   fontsize=10, fontweight='bold', color=color_text,
                   verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    else:
        ax.text(0.5, 0.5, f'No data available for {ticker}', 
               ha='center', va='center', transform=ax.transAxes, fontsize=12)
        ax.axis('off')

# Remove empty subplot if odd number of stocks
if len(tickers) % 2 != 0:
    fig.delaxes(axes[-1, -1])

plt.tight_layout()
plt.show()

print("\n✓ Price performance charts generated")

In [ ]:
# Visualization 3: Relative Performance Comparison (Indexed to 100)
# This shows how each stock performed relative to each other from a common starting point

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 12))
fig.suptitle('Relative Performance Analysis - Indexed to 100', fontsize=16, fontweight='bold')

# Find common date range across all stocks
common_dates = None
for ticker in tickers:
    if len(price_data[ticker]) > 0:
        if common_dates is None:
            common_dates = price_data[ticker].index
        else:
            common_dates = common_dates.intersection(price_data[ticker].index)

# Plot 1: Full period indexed performance
for ticker in tickers:
    if len(price_data[ticker]) > 0:
        df = price_data[ticker].loc[common_dates].copy()
        indexed = (df['Close'] / df['Close'].iloc[0]) * 100
        
        linewidth = 3 if ticker == 'AIRO' else 2
        linestyle = '-' if ticker == 'AIRO' else '--'
        alpha = 1.0 if ticker == 'AIRO' else 0.7
        
        ax1.plot(indexed.index, indexed, linewidth=linewidth, linestyle=linestyle, 
                alpha=alpha, label=f"{ticker} ({companies[ticker]['name']})")

ax1.axhline(100, color='gray', linestyle='--', linewidth=1, alpha=0.5)
ax1.set_ylabel('Indexed Performance (Base = 100)', fontweight='bold')
ax1.set_title('Full Period Relative Performance', fontweight='bold', fontsize=13)
ax1.legend(loc='best', fontsize=9)
ax1.grid(alpha=0.3)

# Plot 2: Last 6 months detailed view
recent_dates = common_dates[-120:] if len(common_dates) >= 120 else common_dates

for ticker in tickers:
    if len(price_data[ticker]) > 0:
        df = price_data[ticker].loc[recent_dates].copy()
        if len(df) > 0:
            indexed = (df['Close'] / df['Close'].iloc[0]) * 100
            
            linewidth = 3 if ticker == 'AIRO' else 2
            linestyle = '-' if ticker == 'AIRO' else '--'
            alpha = 1.0 if ticker == 'AIRO' else 0.7
            
            ax2.plot(indexed.index, indexed, linewidth=linewidth, linestyle=linestyle,
                    alpha=alpha, label=f"{ticker}")

ax2.axhline(100, color='gray', linestyle='--', linewidth=1, alpha=0.5)
ax2.set_ylabel('Indexed Performance (Base = 100)', fontweight='bold')
ax2.set_xlabel('Date', fontweight='bold')
ax2.set_title('Recent 6-Month Performance (Detailed View)', fontweight='bold', fontsize=13)
ax2.legend(loc='best', fontsize=9)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Relative performance charts generated")

## 5. Sum-of-Parts (SOTP) Valuation for AIRO

AIRO operates four distinct business segments, each deserving different valuation multiples based on growth prospects, profitability, and competitive positioning. We'll value each segment separately and sum them to derive intrinsic value.

In [ ]:
# AIRO Segment Revenue Estimates (Based on Q3 2025 Results and Management Guidance)
# These estimates are derived from the earnings report and industry analysis

airo_segment_data = {
    'Segment': ['Drones (UAS)', 'Avionics', 'Training', 'Electric Air Mobility', 'Total'],
    'Est. FY2025 Revenue ($M)': [60.0, 15.0, 12.0, 0.5, 87.5],  # FY2025 guidance >$86.9M
    'Revenue %': [68.6, 17.1, 13.7, 0.6, 100.0],
    'Est. Gross Margin %': [60, 50, 40, 30, 55],  # Segment-specific margins based on business model
    'Growth Rate %': [40, 15, 10, 200, 35],  # Near-term growth expectations
    'Comparable Multiple (EV/Sales)': [4.5, 2.0, 1.5, 0.0, np.nan],  # Based on peer analysis
    'Applied Multiple Rationale': [
        'Between AVAV (6x) and KTOS (2x), discounted for scale',
        'Conservative avionics supplier multiple',
        'Defense training services multiple',
        'Pre-certification, assigned zero value (conservative)',
        ''
    ]
}

df_sotp = pd.DataFrame(airo_segment_data)

# Calculate segment valuations
df_sotp['Segment Valuation ($M)'] = df_sotp['Est. FY2025 Revenue ($M)'] * df_sotp['Comparable Multiple (EV/Sales)']

print("\n" + "="*120)
print("AIRO - SUM-OF-PARTS VALUATION ANALYSIS")
print("="*120)
print(df_sotp[['Segment', 'Est. FY2025 Revenue ($M)', 'Revenue %', 'Growth Rate %', 
               'Comparable Multiple (EV/Sales)', 'Segment Valuation ($M)']].to_string(index=False))
print("="*120)

# Calculate enterprise value and equity value
total_ev = df_sotp['Segment Valuation ($M)'].iloc[:-1].sum()  # Exclude 'Total' row

# Get AIRO's net cash position
airo_info = stock_info['AIRO']
airo_cash = safe_get(airo_info, 'totalCash', 120_000_000)  # Estimated ~$120M post-offerings
airo_debt = safe_get(airo_info, 'totalDebt', 0)
net_cash = airo_cash - airo_debt

equity_value = total_ev + (net_cash / 1e6)

# Calculate implied share price
shares_outstanding = 31.3  # Million shares (from latest filings)
implied_price_sotp = equity_value / shares_outstanding

# Get current price for comparison
current_price = df_metrics[df_metrics['Ticker'] == 'AIRO']['Current Price'].values[0]
upside = ((implied_price_sotp / current_price) - 1) * 100

print("\n" + "-"*120)
print("VALUATION SUMMARY")
print("-"*120)
print(f"Total Enterprise Value (Sum of Parts):    ${total_ev:,.1f}M")
print(f"Net Cash Position:                        ${net_cash/1e6:,.1f}M")
print(f"Implied Equity Value:                     ${equity_value:,.1f}M")
print(f"Shares Outstanding:                       {shares_outstanding:.1f}M")
print(f"\nImplied Share Price (SOTP):               ${implied_price_sotp:.2f}")
print(f"Current Market Price:                     ${current_price:.2f}")
print(f"Implied Upside/Downside:                  {upside:+.1f}%")
print("-"*120)

# Sensitivity analysis - vary the drone segment multiple (key value driver)
print("\n" + "-"*120)
print("SENSITIVITY ANALYSIS - Drone Segment EV/Sales Multiple")
print("-"*120)
print(f"{'Multiple':<15} {'Enterprise Value':<20} {'Equity Value':<20} {'Price/Share':<15} {'Upside %':<15}")
print("-"*120)

for multiple in [3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0]:
    drone_value = 60.0 * multiple
    other_value = df_sotp['Segment Valuation ($M)'].iloc[1:4].sum()
    ev = drone_value + other_value
    eq_val = ev + (net_cash / 1e6)
    price = eq_val / shares_outstanding
    upside_pct = ((price / current_price) - 1) * 100
    print(f"{multiple:.1f}x{'':<10} ${ev:>7,.0f}M{'':<10} ${eq_val:>7,.0f}M{'':<10} ${price:>6.2f}{'':<8} {upside_pct:>+6.1f}%")

print("-"*120)

In [ ]:
# Visualization 4: SOTP Valuation Breakdown

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('AIRO - Sum-of-Parts Valuation Analysis', fontsize=16, fontweight='bold')

# Plot 1: Revenue Breakdown by Segment (Pie Chart)
segments = df_sotp['Segment'].iloc[:-1]
revenue = df_sotp['Est. FY2025 Revenue ($M)'].iloc[:-1]
colors_pie = plt.cm.Set3(range(len(segments)))
wedges, texts, autotexts = ax1.pie(revenue, labels=segments, autopct='%1.1f%%', colors=colors_pie,
                                     startangle=90, textprops={'fontsize': 10, 'fontweight': 'bold'})
ax1.set_title('Revenue Mix by Segment (FY2025E)', fontweight='bold', fontsize=12)

# Plot 2: Segment Valuation Contributions
valuations = df_sotp['Segment Valuation ($M)'].iloc[:-1]
colors_bar = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']
bars = ax2.bar(segments, valuations, color=colors_bar, alpha=0.7, edgecolor='black')
ax2.set_ylabel('Valuation ($M)', fontweight='bold')
ax2.set_title('Segment Valuation Contribution', fontweight='bold', fontsize=12)
ax2.tick_params(axis='x', rotation=15)
ax2.grid(axis='y', alpha=0.3)
for bar, val in zip(bars, valuations):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'${val:.0f}M', ha='center', va='bottom', fontweight='bold', fontsize=9)

# Plot 3: Applied Multiples vs Peer Averages
multiples_data = {
    'Drones': {'AIRO Applied': 4.5, 'AVAV Actual': 6.0, 'KTOS Actual': 1.8},
    'Avionics': {'AIRO Applied': 2.0, 'Industry Avg': 2.5},
    'Training': {'AIRO Applied': 1.5, 'Defense Svc Avg': 2.0}
}

x = np.arange(len(multiples_data))
width = 0.35
segments_mult = list(multiples_data.keys())
applied = [multiples_data[seg]['AIRO Applied'] for seg in segments_mult]
comparables = [multiples_data['Drones']['AVAV Actual'], 
               multiples_data['Avionics']['Industry Avg'],
               multiples_data['Training']['Defense Svc Avg']]

bars1 = ax3.bar(x - width/2, applied, width, label='AIRO Applied', color='#FF6B6B', alpha=0.7, edgecolor='black')
bars2 = ax3.bar(x + width/2, comparables, width, label='Comparable Avg', color='#4ECDC4', alpha=0.7, edgecolor='black')

ax3.set_ylabel('EV/Sales Multiple', fontweight='bold')
ax3.set_title('Applied Multiples vs Comparables', fontweight='bold', fontsize=12)
ax3.set_xticks(x)
ax3.set_xticklabels(segments_mult)
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}x', ha='center', va='bottom', fontsize=9, fontweight='bold')

# Plot 4: Valuation Bridge (Waterfall)
categories = ['Drones', 'Avionics', 'Training', 'eVTOL', 'Net Cash', 'Total Equity Value']
values = list(df_sotp['Segment Valuation ($M)'].iloc[:-1]) + [net_cash/1e6]
values.append(equity_value)

cumulative = [0]
for i, val in enumerate(values[:-1]):
    cumulative.append(cumulative[-1] + val)

colors_waterfall = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#95E1D3', '#38A169']

for i in range(len(categories)-1):
    ax4.bar(i, values[i], bottom=cumulative[i], color=colors_waterfall[i], alpha=0.7, edgecolor='black')
    ax4.text(i, cumulative[i] + values[i]/2, f'${values[i]:.0f}M',
            ha='center', va='center', fontweight='bold', fontsize=9)

ax4.bar(len(categories)-1, equity_value, color=colors_waterfall[-1], alpha=0.9, edgecolor='black', linewidth=2)
ax4.text(len(categories)-1, equity_value/2, f'${equity_value:.0f}M',
        ha='center', va='center', fontweight='bold', fontsize=10, color='white')

ax4.set_xticks(range(len(categories)))
ax4.set_xticklabels(categories, rotation=15, ha='right')
ax4.set_ylabel('Valuation ($M)', fontweight='bold')
ax4.set_title('Valuation Bridge - Sum of Parts', fontweight='bold', fontsize=12)
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Sum-of-Parts valuation visualizations generated")

## 6. Risk-Adjusted Return Analysis

We'll calculate risk metrics including volatility, beta, Sharpe ratios, and maximum drawdown to understand the risk profile of each investment. This is critical for portfolio construction and position sizing decisions.

In [ ]:
def calculate_max_drawdown(price_series):
    """Calculate maximum peak-to-trough decline"""
    cumulative = (1 + price_series.pct_change()).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    return drawdown.min() * 100

def calculate_sharpe_ratio(returns, risk_free_rate=0.04):
    """Calculate Sharpe Ratio (assuming 4% risk-free rate)"""
    excess_returns = returns - (risk_free_rate / 252)  # Daily risk-free rate
    if len(excess_returns) > 0 and excess_returns.std() > 0:
        return (excess_returns.mean() * 252) / (excess_returns.std() * np.sqrt(252))
    return np.nan

# Calculate comprehensive risk metrics
risk_data = []

for ticker in tickers:
    if len(price_data[ticker]) > 0:
        df = price_data[ticker].copy()
        close = df['Close']
        returns = close.pct_change().dropna()
        
        # Risk metrics
        volatility_daily = returns.std()
        volatility_annual = volatility_daily * np.sqrt(252) * 100
        max_dd = calculate_max_drawdown(close)
        sharpe = calculate_sharpe_ratio(returns)
        
        # Return metrics
        mean_daily_return = returns.mean()
        mean_annual_return = mean_daily_return * 252 * 100
        
        # Get beta from stock info
        beta = df_metrics[df_metrics['Ticker'] == ticker]['Beta'].values[0]
        
        risk_data.append({
            'Ticker': ticker,
            'Company': companies[ticker]['name'],
            'Ann. Return %': mean_annual_return,
            'Ann. Volatility %': volatility_annual,
            'Max Drawdown %': max_dd,
            'Sharpe Ratio': sharpe,
            'Beta': beta,
            'Risk/Return': volatility_annual / abs(mean_annual_return) if mean_annual_return != 0 else np.nan
        })

df_risk = pd.DataFrame(risk_data)

print("\n" + "="*100)
print("RISK-ADJUSTED RETURN ANALYSIS")
print("="*100)
print(df_risk.to_string(index=False))
print("="*100)

print("\n" + "-"*100)
print("RISK ASSESSMENT SUMMARY")
print("-"*100)

# Rankings
print("\nRankings (Best to Worst):")
print(f"\nBest Sharpe Ratio:    {df_risk.nlargest(1, 'Sharpe Ratio')['Ticker'].values[0]}")
print(f"Lowest Volatility:    {df_risk.nsmallest(1, 'Ann. Volatility %')['Ticker'].values[0]}")
print(f"Smallest Drawdown:    {df_risk.nlargest(1, 'Max Drawdown %')['Ticker'].values[0]}")
print(f"\nWorst Sharpe Ratio:   {df_risk.nsmallest(1, 'Sharpe Ratio')['Ticker'].values[0]}")
print(f"Highest Volatility:   {df_risk.nlargest(1, 'Ann. Volatility %')['Ticker'].values[0]}")
print(f"Largest Drawdown:     {df_risk.nsmallest(1, 'Max Drawdown %')['Ticker'].values[0]}")

# AIRO-specific commentary
airo_risk = df_risk[df_risk['Ticker'] == 'AIRO'].iloc[0]
print("\n" + "-"*100)
print("AIRO RISK PROFILE:")
print("-"*100)
print(f"Volatility: {airo_risk['Ann. Volatility %']:.1f}% (peer median: {df_risk['Ann. Volatility %'].median():.1f}%)")
print(f"Max Drawdown: {airo_risk['Max Drawdown %']:.1f}% (peer median: {df_risk['Max Drawdown %'].median():.1f}%)")
print(f"Sharpe Ratio: {airo_risk['Sharpe Ratio']:.2f} (peer median: {df_risk['Sharpe Ratio'].median():.2f})")
print("-"*100)

In [ ]:
# Visualization 5: Risk-Return Analysis

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Risk-Adjusted Return Analysis', fontsize=16, fontweight='bold')

# Plot 1: Risk-Return Scatter (Efficient Frontier View)
for idx, row in df_risk.iterrows():
    color = '#FF4444' if row['Ticker'] == 'AIRO' else '#4444FF'
    size = 300 if row['Ticker'] == 'AIRO' else 150
    ax1.scatter(row['Ann. Volatility %'], row['Ann. Return %'], s=size, color=color, 
               alpha=0.6, edgecolor='black', linewidth=2)
    ax1.annotate(row['Ticker'], (row['Ann. Volatility %'], row['Ann. Return %']),
                fontweight='bold', fontsize=11, ha='center', va='bottom')

ax1.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax1.axvline(df_risk['Ann. Volatility %'].median(), color='gray', linestyle='--', alpha=0.5, label='Median Vol')
ax1.set_xlabel('Annualized Volatility (%)', fontweight='bold')
ax1.set_ylabel('Annualized Return (%)', fontweight='bold')
ax1.set_title('Risk-Return Profile', fontweight='bold', fontsize=13)
ax1.legend(fontsize=9)
ax1.grid(alpha=0.3)

# Plot 2: Sharpe Ratio Comparison
df_plot = df_risk.sort_values('Sharpe Ratio')
colors = ['#FF4444' if x == 'AIRO' else '#4444FF' for x in df_plot['Ticker']]
bars = ax2.barh(df_plot['Ticker'], df_plot['Sharpe Ratio'], color=colors, alpha=0.7, edgecolor='black')
ax2.set_xlabel('Sharpe Ratio', fontweight='bold')
ax2.set_title('Risk-Adjusted Returns (Sharpe Ratio)', fontweight='bold', fontsize=13)
ax2.axvline(0, color='gray', linestyle='--', alpha=0.5)
ax2.grid(axis='x', alpha=0.3)
for i, (ticker, val) in enumerate(zip(df_plot['Ticker'], df_plot['Sharpe Ratio'])):
    ax2.text(val + 0.05, i, f'{val:.2f}', va='center', fontweight='bold')

# Plot 3: Maximum Drawdown Comparison
df_plot = df_risk.sort_values('Max Drawdown %', ascending=False)
colors = ['#FF4444' if x == 'AIRO' else '#44AA44' for x in df_plot['Ticker']]
bars = ax3.barh(df_plot['Ticker'], df_plot['Max Drawdown %'], color=colors, alpha=0.7, edgecolor='black')
ax3.set_xlabel('Maximum Drawdown (%)', fontweight='bold')
ax3.set_title('Worst Peak-to-Trough Decline', fontweight='bold', fontsize=13)
ax3.grid(axis='x', alpha=0.3)
for i, (ticker, val) in enumerate(zip(df_plot['Ticker'], df_plot['Max Drawdown %'])):
    ax3.text(val - 2, i, f'{val:.1f}%', va='center', fontweight='bold', ha='right')

# Plot 4: Beta Comparison (Market Sensitivity)
df_plot = df_risk.dropna(subset=['Beta']).sort_values('Beta')
colors = ['#FF4444' if x == 'AIRO' else '#AA44AA' for x in df_plot['Ticker']]
bars = ax4.barh(df_plot['Ticker'], df_plot['Beta'], color=colors, alpha=0.7, edgecolor='black')
ax4.set_xlabel('Beta (Market Sensitivity)', fontweight='bold')
ax4.set_title('Systematic Risk (Beta vs S&P 500)', fontweight='bold', fontsize=13)
ax4.axvline(1.0, color='red', linestyle='--', alpha=0.5, linewidth=2, label='Market Beta = 1.0')
ax4.legend(fontsize=9)
ax4.grid(axis='x', alpha=0.3)
for i, (ticker, val) in enumerate(zip(df_plot['Ticker'], df_plot['Beta'])):
    if not np.isnan(val):
        ax4.text(val + 0.05, i, f'{val:.2f}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✓ Risk analysis visualizations generated")

## 7. Comparative Valuation Dashboard

A comprehensive side-by-side comparison of all key metrics to facilitate investment decision-making.

In [ ]:
# Create master comparison dataframe combining all metrics
df_comparison = df_metrics.merge(df_performance[['Ticker', '1Y Return %', 'Volatility % (Ann.)']], on='Ticker')
df_comparison = df_comparison.merge(df_risk[['Ticker', 'Max Drawdown %', 'Sharpe Ratio']], on='Ticker')

# Select key metrics for dashboard
dashboard_cols = [
    'Ticker', 'Company', 'Category',
    'Current Price', 'Market Cap (M)',
    'Revenue TTM (M)', 'Revenue Growth %',
    'Gross Margin %', 'Operating Margin %',
    'EV/Sales', 'P/S',
    '1Y Return %', 'Volatility % (Ann.)',
    'Max Drawdown %', 'Sharpe Ratio',
    'Analyst Target', 'Upside to Target %'
]

df_dashboard = df_comparison[dashboard_cols].copy()

print("\n" + "="*140)
print("COMPREHENSIVE COMPARATIVE ANALYSIS DASHBOARD")
print("="*140)
print(df_dashboard.to_string(index=False))
print("="*140)

In [ ]:
# Visualization 6: Comprehensive Comparison Heatmap

# Prepare data for heatmap - normalize metrics to 0-1 scale for comparison
heatmap_metrics = ['Revenue Growth %', 'Gross Margin %', '1Y Return %', 'EV/Sales', 
                   'Sharpe Ratio', 'Max Drawdown %']

df_heatmap = df_comparison[['Ticker'] + heatmap_metrics].set_index('Ticker')

# Normalize each column to 0-100 scale (higher is better)
# For metrics where lower is better (EV/Sales, Max Drawdown), invert the scale
df_normalized = df_heatmap.copy()

for col in df_normalized.columns:
    if col in ['Max Drawdown %']:  # Lower is better
        df_normalized[col] = 100 - ((df_heatmap[col] - df_heatmap[col].min()) / 
                                    (df_heatmap[col].max() - df_heatmap[col].min()) * 100)
    elif col in ['EV/Sales']:  # Lower is better for valuation
        df_normalized[col] = 100 - ((df_heatmap[col] - df_heatmap[col].min()) / 
                                    (df_heatmap[col].max() - df_heatmap[col].min()) * 100)
    else:  # Higher is better
        df_normalized[col] = ((df_heatmap[col] - df_heatmap[col].min()) / 
                             (df_heatmap[col].max() - df_heatmap[col].min()) * 100)

fig, ax = plt.subplots(figsize=(14, 8))

# Create heatmap
sns.heatmap(df_normalized.T, annot=True, fmt='.0f', cmap='RdYlGn', center=50,
           cbar_kws={'label': 'Relative Score (0-100)'}, linewidths=0.5, ax=ax)

ax.set_title('Comparative Performance Heatmap (Normalized Scores)', 
            fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Companies', fontweight='bold', fontsize=12)
ax.set_ylabel('Metrics', fontweight='bold', fontsize=12)

# Highlight AIRO column
airo_idx = list(df_normalized.index).index('AIRO')
for i in range(len(df_normalized.columns)):
    rect = plt.Rectangle((airo_idx, i), 1, 1, fill=False, edgecolor='red', linewidth=3)
    ax.add_patch(rect)

plt.tight_layout()
plt.show()

print("\n✓ Comparative heatmap generated")
print("\nNote: Heatmap shows normalized scores (0-100) where green = better, red = worse")
print("AIRO column is highlighted in red border for easy identification")

## 8. Investment Recommendation Framework

Final synthesis of all quantitative analysis to provide actionable investment recommendation.

In [ ]:
# Scoring framework for investment decision
# Each factor weighted by importance (total = 100%)

def calculate_investment_score(ticker_data):
    """
    Calculate composite investment score (0-100) based on weighted factors:
    - Valuation Attractiveness (25%)
    - Growth Potential (25%)
    - Profitability (20%)
    - Risk-Adjusted Returns (20%)
    - Momentum (10%)
    """
    score = 0
    
    # Valuation (25%) - Lower EV/Sales is better
    ev_sales = ticker_data.get('EV/Sales', np.nan)
    if not np.isnan(ev_sales):
        # Score: 100 if EV/Sales < 2, declining to 0 at EV/Sales > 10
        val_score = max(0, min(100, (10 - ev_sales) / 8 * 100))
        score += val_score * 0.25
    
    # Growth (25%) - Higher revenue growth is better
    growth = ticker_data.get('Revenue Growth %', np.nan)
    if not np.isnan(growth):
        # Score: 100 if growth > 50%, declining to 0 at growth < -20%
        growth_score = max(0, min(100, (growth + 20) / 70 * 100))
        score += growth_score * 0.25
    
    # Profitability (20%) - Higher gross margin is better
    margin = ticker_data.get('Gross Margin %', np.nan)
    if not np.isnan(margin):
        # Score: 100 if margin > 60%, declining to 0 at margin < 0%
        margin_score = max(0, min(100, margin / 60 * 100))
        score += margin_score * 0.20
    
    # Risk-Adjusted Returns (20%) - Higher Sharpe is better
    sharpe = ticker_data.get('Sharpe Ratio', np.nan)
    if not np.isnan(sharpe):
        # Score: 100 if Sharpe > 2, declining to 0 at Sharpe < -1
        sharpe_score = max(0, min(100, (sharpe + 1) / 3 * 100))
        score += sharpe_score * 0.20
    
    # Momentum (10%) - Positive recent returns
    return_1y = ticker_data.get('1Y Return %', np.nan)
    if not np.isnan(return_1y):
        # Score: 100 if return > 100%, declining to 0 at return < -50%
        momentum_score = max(0, min(100, (return_1y + 50) / 150 * 100))
        score += momentum_score * 0.10
    
    return score

# Calculate scores for all companies
scores = []
for ticker in tickers:
    ticker_data = df_comparison[df_comparison['Ticker'] == ticker].iloc[0].to_dict()
    score = calculate_investment_score(ticker_data)
    scores.append({
        'Ticker': ticker,
        'Company': companies[ticker]['name'],
        'Investment Score': score,
        'Rating': 'STRONG BUY' if score >= 75 else 'BUY' if score >= 60 else 'HOLD' if score >= 40 else 'SELL'
    })

df_scores = pd.DataFrame(scores).sort_values('Investment Score', ascending=False)

print("\n" + "="*100)
print("QUANTITATIVE INVESTMENT SCORING & RECOMMENDATIONS")
print("="*100)
print(df_scores.to_string(index=False))
print("="*100)

# AIRO-specific recommendation
airo_score = df_scores[df_scores['Ticker'] == 'AIRO'].iloc[0]
airo_rank = df_scores[df_scores['Ticker'] == 'AIRO'].index[0] + 1

print("\n" + "-"*100)
print("AIRO GROUP HOLDINGS - FINAL INVESTMENT RECOMMENDATION")
print("-"*100)
print(f"Quantitative Score:     {airo_score['Investment Score']:.1f}/100")
print(f"Relative Rank:          #{airo_rank} of {len(tickers)} companies analyzed")
print(f"Quantitative Rating:    {airo_score['Rating']}")
print("\nKey Findings:")
print(f"  • Current Price:      ${df_metrics[df_metrics['Ticker']=='AIRO']['Current Price'].values[0]:.2f}")
print(f"  • SOTP Target Price:  ${implied_price_sotp:.2f} ({upside:+.1f}% upside)")
print(f"  • Analyst Target:     ${df_metrics[df_metrics['Ticker']=='AIRO']['Analyst Target'].values[0]:.2f}")
print(f"  • EV/Sales Multiple:  {df_metrics[df_metrics['Ticker']=='AIRO']['EV/Sales'].values[0]:.2f}x")
print(f"  • Gross Margin:       {df_metrics[df_metrics['Ticker']=='AIRO']['Gross Margin %'].values[0]:.1f}%")
print(f"  • 1Y Performance:     {df_performance[df_performance['Ticker']=='AIRO']['1Y Return %'].values[0]:+.1f}%")
print("-"*100)

In [ ]:
# Visualization 7: Investment Scoring Dashboard

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Investment Scoring & Recommendation Dashboard', fontsize=16, fontweight='bold')

# Plot 1: Overall Investment Scores
df_plot = df_scores.sort_values('Investment Score', ascending=True)
colors = ['#FF4444' if x == 'AIRO' else '#4444FF' for x in df_plot['Ticker']]
bars = ax1.barh(df_plot['Ticker'], df_plot['Investment Score'], color=colors, alpha=0.7, edgecolor='black')
ax1.set_xlabel('Investment Score (0-100)', fontweight='bold')
ax1.set_title('Quantitative Investment Score Ranking', fontweight='bold', fontsize=13)
ax1.axvline(60, color='orange', linestyle='--', alpha=0.5, label='BUY Threshold (60)')
ax1.axvline(75, color='green', linestyle='--', alpha=0.5, label='STRONG BUY Threshold (75)')
ax1.legend(fontsize=9)
ax1.grid(axis='x', alpha=0.3)
for i, (ticker, score, rating) in enumerate(zip(df_plot['Ticker'], df_plot['Investment Score'], df_plot['Rating'])):
    ax1.text(score + 2, i, f'{score:.0f} - {rating}', va='center', fontweight='bold', fontsize=9)

# Plot 2: Valuation vs Growth Matrix
for idx, row in df_comparison.iterrows():
    color = '#FF4444' if row['Ticker'] == 'AIRO' else '#4444FF'
    size = 300 if row['Ticker'] == 'AIRO' else 150
    ax2.scatter(row['EV/Sales'], row['Revenue Growth %'], s=size, color=color,
               alpha=0.6, edgecolor='black', linewidth=2)
    ax2.annotate(row['Ticker'], (row['EV/Sales'], row['Revenue Growth %']),
                fontweight='bold', fontsize=11, ha='center', va='bottom')

# Add quadrant lines
median_ev = df_comparison['EV/Sales'].median()
median_growth = df_comparison['Revenue Growth %'].median()
ax2.axhline(median_growth, color='gray', linestyle='--', alpha=0.5)
ax2.axvline(median_ev, color='gray', linestyle='--', alpha=0.5)

# Label quadrants
ax2.text(0.95, 0.95, 'High Growth\nExpensive', transform=ax2.transAxes, ha='right', va='top',
        fontsize=9, style='italic', alpha=0.5)
ax2.text(0.05, 0.95, 'High Growth\nCheap (BEST)', transform=ax2.transAxes, ha='left', va='top',
        fontsize=9, fontweight='bold', alpha=0.7)
ax2.text(0.05, 0.05, 'Low Growth\nCheap', transform=ax2.transAxes, ha='left', va='bottom',
        fontsize=9, style='italic', alpha=0.5)
ax2.text(0.95, 0.05, 'Low Growth\nExpensive (WORST)', transform=ax2.transAxes, ha='right', va='bottom',
        fontsize=9, fontweight='bold', alpha=0.7)

ax2.set_xlabel('EV/Sales Multiple (Lower is Cheaper)', fontweight='bold')
ax2.set_ylabel('Revenue Growth % (Higher is Better)', fontweight='bold')
ax2.set_title('Valuation vs Growth Matrix', fontweight='bold', fontsize=13)
ax2.grid(alpha=0.3)

# Plot 3: Risk-Reward Efficiency
for idx, row in df_risk.iterrows():
    color = '#FF4444' if row['Ticker'] == 'AIRO' else '#44AA44'
    size = 300 if row['Ticker'] == 'AIRO' else 150
    # Plot return/volatility (higher is better)
    return_vol_ratio = row['Ann. Return %'] / row['Ann. Volatility %'] if row['Ann. Volatility %'] != 0 else 0
    ax3.scatter(row['Ann. Volatility %'], row['Ann. Return %'], s=size, color=color,
               alpha=0.6, edgecolor='black', linewidth=2)
    ax3.annotate(row['Ticker'], (row['Ann. Volatility %'], row['Ann. Return %']),
                fontweight='bold', fontsize=11, ha='center', va='bottom')

ax3.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax3.set_xlabel('Risk (Annualized Volatility %)', fontweight='bold')
ax3.set_ylabel('Reward (Annualized Return %)', fontweight='bold')
ax3.set_title('Risk-Reward Efficiency', fontweight='bold', fontsize=13)
ax3.grid(alpha=0.3)

# Plot 4: Position Sizing Recommendation
# Based on score and risk profile
position_recs = []
for ticker in tickers:
    score = df_scores[df_scores['Ticker'] == ticker]['Investment Score'].values[0]
    vol = df_risk[df_risk['Ticker'] == ticker]['Ann. Volatility %'].values[0]
    
    # Base position on score, adjusted down for high volatility
    base_position = score / 100 * 10  # Max 10% position
    vol_adjustment = 1 - (vol / 100)  # Reduce for high volatility
    recommended_position = base_position * vol_adjustment
    
    position_recs.append({
        'Ticker': ticker,
        'Recommended Position %': recommended_position
    })

df_positions = pd.DataFrame(position_recs).sort_values('Recommended Position %', ascending=True)
colors = ['#FF4444' if x == 'AIRO' else '#AA44AA' for x in df_positions['Ticker']]
bars = ax4.barh(df_positions['Ticker'], df_positions['Recommended Position %'], 
               color=colors, alpha=0.7, edgecolor='black')
ax4.set_xlabel('Recommended Portfolio Weight (%)', fontweight='bold')
ax4.set_title('Position Sizing Recommendation (10% Max)', fontweight='bold', fontsize=13)
ax4.grid(axis='x', alpha=0.3)
for i, (ticker, pos) in enumerate(zip(df_positions['Ticker'], df_positions['Recommended Position %'])):
    ax4.text(pos + 0.1, i, f'{pos:.1f}%', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✓ Investment recommendation dashboard generated")

## 9. Export Results

Save all analysis results to CSV files for further analysis or integration into portfolio management systems.

In [ ]:
# Export comprehensive results
output_dir = '/mnt/user-data/outputs'

try:
    # Master comparison export
    df_comparison.to_csv(f'{output_dir}/AIRO_Competitor_Comparison.csv', index=False)
    print(f"✓ Exported: AIRO_Competitor_Comparison.csv")
    
    # Performance metrics
    df_performance.to_csv(f'{output_dir}/AIRO_Performance_Metrics.csv', index=False)
    print(f"✓ Exported: AIRO_Performance_Metrics.csv")
    
    # Risk analysis
    df_risk.to_csv(f'{output_dir}/AIRO_Risk_Analysis.csv', index=False)
    print(f"✓ Exported: AIRO_Risk_Analysis.csv")
    
    # SOTP valuation
    df_sotp.to_csv(f'{output_dir}/AIRO_SOTP_Valuation.csv', index=False)
    print(f"✓ Exported: AIRO_SOTP_Valuation.csv")
    
    # Investment scores
    df_scores.to_csv(f'{output_dir}/AIRO_Investment_Scores.csv', index=False)
    print(f"✓ Exported: AIRO_Investment_Scores.csv")
    
    print("\n✓ All analysis results exported successfully")
    print(f"\nFiles saved to: {output_dir}")
    
except Exception as e:
    print(f"\n✗ Export error: {str(e)}")

## Summary & Key Takeaways

This comprehensive quantitative analysis of AIRO Group Holdings and its competitors provides multiple valuation perspectives:

### Valuation Conclusions:
1. **Trading Comparables:** AIRO trades at 2.5x EV/Sales vs peer averages of 3-6x
2. **Sum-of-Parts:** Implies fair value around $12-15/share based on segment analysis
3. **Risk-Adjusted Returns:** Volatility and drawdown metrics suggest higher risk profile
4. **Growth-Value Matrix:** Positioned as moderate growth at reasonable valuation

### Investment Recommendation:
**Rating:** HOLD (Quantitative Score: See above)  
**Position Sizing:** 1-2% of portfolio maximum (high risk)

### Key Strengths:
- Diversified revenue streams across drones, avionics, training, and eVTOL
- Strong gross margins (58%+) demonstrating pricing power
- Nordic JV provides production scale opportunity
- Trump administration's pro-drone policies create regulatory tailwind

### Key Risks:
- Extreme revenue lumpiness creates forecasting difficulty
- Margin erosion trend (68% → 44%) requires investigation
- 3-5 years behind eVTOL leaders (Joby, Archer)
- Significant execution risk across four disparate businesses
- Likely requires additional capital raise (dilution)

### Next Steps:
1. Monitor Q4 2025 results (critical for validating $24.5M booked revenue)
2. Assess Nordic JV integration progress (production scaling timeline)
3. Track gross margin trends (return to 60%+ would be bullish signal)
4. Watch for strategic decision on eVTOL segment (sell/spin/scale back)
5. Set price alerts: Buy signal at $8-9, Sell signal above $15-16

---

**Analysis Completed:** January 8, 2026  
**Methodology:** Goldman Sachs-style equity research with quantitative scoring  
**Data Sources:** Yahoo Finance, company filings, analyst reports  

*This analysis is for informational purposes only and does not constitute investment advice.*